In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

from sklearn.metrics import mean_squared_error

In [2]:
CONFIG = {
    'TRAIN_PATH': '/kaggle/input/datathon-entel-2022-reto2/train.csv',
    'TEST_PATH': '/kaggle/input/datathon-entel-2022-reto2/test.csv',
    'SAMPLE_SUBMISSION': '/kaggle/input/datathon-entel-2022-reto2/test_sample.csv'
}

df_train = pd.read_csv(CONFIG['TRAIN_PATH'])
df_test = pd.read_csv(CONFIG['TEST_PATH'])
df_sub = pd.read_csv(CONFIG['SAMPLE_SUBMISSION'])

# Cantidad de celulares vendidos por cada semana

In [3]:
px.area(df_train.iloc[: , 5:].sum())

# ¿Cómo está distribuido los datos en las últimas 10 semanas?

In [4]:
px.scatter(df_train.iloc[:, -10:].stack().value_counts(normalize=True))

#### 84.5% de los datos son valores "0" 
#### 7.19% de los datos son valores "1"
#### 3.00% de los datos son valores "2" 
#### 1.40% de los datos son valores "3"
#### Estos solo 4 datos represetan el 96.17% de todo el conjunto

# ¿Existe relación entre las 10 semanas previa con la siguientes semana 10?

In [5]:
def previous_rmse(start, end):
    print(f'START {start} - END {end}')
    time = end - start
    a1 = df_train.iloc[:, start: end].stack().reset_index(drop=True)
    a2 = df_train.iloc[:, end: end + time].stack().reset_index(drop=True)
    rmse = mean_squared_error(a1, a2, squared=False)
    return rmse

In [6]:
print(previous_rmse(5, 15))
print(previous_rmse(15, 25))
print(previous_rmse(25, 35))
print(previous_rmse(35, 45))

START 5 - END 15
5.657942955462724
START 15 - END 25
6.427028600162597
START 25 - END 35
6.2113736768435
START 35 - END 45
5.466776992892005


# ¿Cómo ha variado los datos con respecto a la semana previa y la semana próxima?

In [7]:
rmse_list = []
for i in range(5, 45):
    _rmse =previous_rmse(i, i + 1)
    rmse_list.append(_rmse)

START 5 - END 6
START 6 - END 7
START 7 - END 8
START 8 - END 9
START 9 - END 10
START 10 - END 11
START 11 - END 12
START 12 - END 13
START 13 - END 14
START 14 - END 15
START 15 - END 16
START 16 - END 17
START 17 - END 18
START 18 - END 19
START 19 - END 20
START 20 - END 21
START 21 - END 22
START 22 - END 23
START 23 - END 24
START 24 - END 25
START 25 - END 26
START 26 - END 27
START 27 - END 28
START 28 - END 29
START 29 - END 30
START 30 - END 31
START 31 - END 32
START 32 - END 33
START 33 - END 34
START 34 - END 35
START 35 - END 36
START 36 - END 37
START 37 - END 38
START 38 - END 39
START 39 - END 40
START 40 - END 41
START 41 - END 42
START 42 - END 43
START 43 - END 44
START 44 - END 45


In [8]:
px.line(rmse_list)

In [9]:
px.line(pd.Series(rmse_list).diff())

#### Como podemos observar en las últimas semanas la variación entre las semanas se ha ido reduciendo
#### Si queremos predicer la siguiente semana, la semana previa sería una gran variable

In [10]:
p = 0.99
x_base = 100

print(df_train.iloc[:, 5:15].stack().quantile(p))
print(df_train.iloc[:, 15:25].stack().quantile(p))
print(df_train.iloc[:, 25:35].stack().quantile(p))
print(df_train.iloc[:, 35:45].stack().quantile(p))
print(df_train.iloc[:, 45:55].stack().quantile(p))

15.0
14.0
15.0
14.0
11.0


#### Entre todas las semanas el quantil 99 nunca llega superar el valor 15

#### Solo un 1% de todos los valores a predecir son mayores a el valor 15

# ¿Cómo es la distribución en cada semana?

In [11]:
print(df_train.iloc[:, 5:15].stack().value_counts(normalize=True)[:5])
print('*' * 50)
print(df_train.iloc[:, 15:25].stack().value_counts(normalize=True)[:5])
print('*' * 50)
print(df_train.iloc[:, 25:35].stack().value_counts(normalize=True)[:5])
print('*' * 50)
print(df_train.iloc[:, 35:45].stack().value_counts(normalize=True)[:5])
print('*' * 50)
print(df_train.iloc[:, 45:55].stack().value_counts(normalize=True)[:5])

0    0.824529
1    0.077616
2    0.033125
3    0.017001
4    0.010279
dtype: float64
**************************************************
0    0.834013
1    0.071893
2    0.031009
3    0.016639
4    0.010372
dtype: float64
**************************************************
0    0.831355
1    0.073131
2    0.030458
3    0.016567
4    0.010167
dtype: float64
**************************************************
0    0.839934
1    0.069648
2    0.030155
3    0.015600
4    0.009569
dtype: float64
**************************************************
0    0.845486
1    0.071948
2    0.029377
3    0.014983
4    0.009181
dtype: float64


#### Como observamos, en todas las semanas más del 80% de los datos son 0, mientras que de un 15 a 20 % son mayores de 1

# Analizando los cambios en base a las 10 primeras semanas con respecto a las semanas del 10 al 20

In [12]:
df_train.iloc[:, 5:15].sum(axis=1).value_counts(normalize=True)

0       0.572637
1       0.091451
2       0.060819
3       0.037776
4       0.029169
          ...   
2922    0.000021
265     0.000021
468     0.000021
553     0.000021
606     0.000021
Length: 418, dtype: float64

In [13]:
df_train[(df_train.iloc[:, 5:15].sum(axis=1) == 0)].iloc[:, 15:25].sum(axis=1).value_counts(normalize=True)[:5]

0    0.834931
1    0.077222
2    0.035390
3    0.017066
4    0.009958
dtype: float64

#### En las 10 primeras semanas el 57% de los registros tienen como suma total de 0 y el 83.49% del 57% seguirán teniendo un total de 0 ventas en la siguientes 10 semanas

In [14]:
df_train[(df_train.iloc[:, 5:15].sum(axis=1) > 0)].iloc[:, 15:25].sum(axis=1).value_counts(normalize=True)[:5]

0    0.278621
1    0.117609
2    0.077629
3    0.055506
4    0.044246
dtype: float64

#### Mientras que del 43% de los registros que venden al menos uno, el 27.86% no venderán ningun celular

# Analizando los cambios en base a las semanas del 10 al 20 con respecto a la semanas del 20 al 30

In [15]:
df_train.iloc[:, 15:25].sum(axis=1).value_counts(normalize=True)

0      0.597185
1      0.094482
2      0.053442
3      0.033494
4      0.024612
         ...   
317    0.000021
834    0.000021
726    0.000021
407    0.000021
421    0.000021
Length: 372, dtype: float64

#### En el segundo grupo de 10 semanas, el 60% de los registros no llegan a vender ningún celular

In [16]:
df_train[(df_train.iloc[:, 15:25].sum(axis=1) == 0)].iloc[:, 25:35].sum(axis=1).value_counts(normalize=True)[:5]

0    0.696851
1    0.105286
2    0.047602
3    0.028433
4    0.018601
dtype: float64

#### En las 10 primeras semanas el 59.71% de los registros tienen como suma total de 0 y el 69.68% del 59.71% seguirán teniendo un total de 0 ventas en la siguientes 10 semanas

In [17]:
df_train[(df_train.iloc[:, 15:25].sum(axis=1) > 0)].iloc[:, 25:35].sum(axis=1).value_counts(normalize=True)[:5]

0    0.346490
1    0.113830
2    0.082044
3    0.058310
4    0.043785
dtype: float64

#### Mientras que del 40% de los registros que venden al menos uno, el 34.64% no venderán ningun celular

# Analizando los cambios en base a las semanas del 30 al 40 con respecto a la semanas del 40 al 50

In [18]:
df_train.iloc[:, 35:45].sum(axis=1).value_counts(normalize=True)

0      0.587900
1      0.086363
2      0.052890
3      0.035677
4      0.026625
         ...   
436    0.000021
704    0.000021
309    0.000021
423    0.000021
306    0.000021
Length: 363, dtype: float64

#### Similar a lo que ha pasado anteriomente, el 58.79% de los registros tiene unas ventas totales de 0 celulares en las semanas 30 y 40

In [19]:
df_train[(df_train.iloc[:, 35:45].sum(axis=1) == 0)].iloc[:, 45:55].sum(axis=1).value_counts(normalize=True)[:5]

0    0.785490
1    0.081636
2    0.036996
3    0.020084
4    0.012512
dtype: float64

#### De los 58.79% de los datos solo un poco más del 20% a logrado vender uno o más celuares mientras que casi el 80% de los celulares que no vendieron ningún celular no venderán ninguno en las siguiente 10 semanas

In [20]:
df_train[(df_train.iloc[:, 35:45].sum(axis=1) > 0)].iloc[:, 45:55].sum(axis=1).value_counts(normalize=True)[:5]

0    0.321039
1    0.115484
2    0.084002
3    0.056430
4    0.046759
dtype: float64

#### Del 20% que logró vender un celular en las 10 semanas anteriores el 32% no venderá ninguno en la siguiente 10 semanas

##### Como se puede entender:
* Casi el 60 o 70 % de los registros serán 0
* De lo anterior la mayoría (70%) aproximandamente serán 0, es decir casi un 50% de los datos totales serán 0

# Analizando los registros en base a sus ventas de la última semana

In [21]:
df_train_c = df_train.copy()
df_train_c['TOTAL'] = df_train_c.iloc[:, 35:45].sum(axis=1)
df_train_c = df_train_c.sort_values(by='TOTAL')

In [22]:
df_train_c['TOTAL'].value_counts(normalize=True)

0       0.587900
1       0.086363
2       0.052890
3       0.035677
4       0.026625
          ...   
318     0.000021
321     0.000021
322     0.000021
323     0.000021
4364    0.000021
Name: TOTAL, Length: 363, dtype: float64

#### Como se refleja en el resultado anterior el 58.79% de los registros o filas tienen un total de ventas de 0 celulares en las últimas 10 semanas

In [23]:
px.line(df_train_c.T.iloc[5:-1, -10:])

In [24]:
_index = 23408

In [25]:
px.line(df_train.iloc[_index, 5:55])

In [26]:
px.line(df_train.iloc[_index, 5:55].diff())

#### De la gráfica anterior se puede terminar que en las primeras 20 semanas no se logró vender a este producto en el tal punto de venta. Puede ser porque anteriormentente no había este producto o todavía no llegaba al local a vender.

In [27]:
_index = 3270

In [28]:
px.line(df_train.iloc[_index, 5:55])

In [29]:
px.line(df_train.iloc[_index, 5:55].diff())

#### A diferencia del anterior registro, la venta de este producto se fue deluyendo con el tiempo. Puede que sea debido a que ya tenía un tiempo en el mercado o que haya salido un nuevo producto de este modelo - gama.

# Relación entre la variable de volatilidad y el objetivo

In [30]:
df_train_c['volatility'] = np.sqrt((
        np.log(
            df_train.iloc[:, 35:45].T.ewm(1).mean()
        ).stack().apply(lambda x: x if str(x) != '-inf' else 0).unstack(level=1)
    ).sum())

/opt/conda/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning:

invalid value encountered in sqrt



#### La volatilidad es una medida para determinar la variación que tiene cierto producto sobre su precio. Una volatilidad alta significa que el producto varía mucho, una volatilidad baja significa que el producto no varia mucho sus (ventas). 

In [31]:
px.scatter(df_train_c, y='TOTAL', x='volatility')

#### La volatilidad en las últimas 10 semanas puede ser una variable muy relacionada al objetivo, si asumimos que la suma de las ventas en las últimas semans sea nuestro objetivo.